# **Assignment # 3**

#Humayun Malik
#21I-2562
#Section:D

#**Word2Vec Notebook**

In [29]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import resample
import numpy as np
import random
import nltk


# Load Data

In [30]:

# Load dataset
from google.colab import files

uploaded = files.upload()  # This opens a file upload dialog


Saving spam_or_not_spam.csv to spam_or_not_spam (2).csv


In [31]:

import pandas as pd

# Load the uploaded CSV file
file_path = "spam_or_not_spam.csv"
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(df.head())


                                               email  label
0   date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...      0
1  martin a posted tassos papadopoulos the greek ...      0
2  man threatens explosion in moscow thursday aug...      0
3  klez the virus that won t die already the most...      0
4   in adding cream to spaghetti carbonara which ...      0


#Pre-Processing


In [32]:

# Ensure that all values in the 'email' column are strings
df['email'] = df['email'].astype(str)

# Remove unnecessary characters
df['email'] = df['email'].apply(lambda x: re.sub(r'[^\w\s]', '', x))  # Remove punctuation
df['email'] = df['email'].apply(lambda x: re.sub(r'\d+', '', x))  # Remove numbers

# Convert to lowercase
df['email'] = df['email'].str.lower()


# Display the first few rows of the dataset
print(df.head())


                                               email  label
0   date wed number aug number number number numb...      0
1  martin a posted tassos papadopoulos the greek ...      0
2  man threatens explosion in moscow thursday aug...      0
3  klez the virus that won t die already the most...      0
4   in adding cream to spaghetti carbonara which ...      0


In [33]:
import nltk
nltk.download('punkt_tab')
# Tokenize text
df['tokens'] = df['email'].apply(word_tokenize)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [34]:
import nltk
nltk.download('punkt')
nltk.download('stopwords') # Download the stopwords resource

# Remove stopwords
from nltk.corpus import stopwords
# Remove stopwords
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
!pip install nltk
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
# Perform lemmatization
lemmatizer = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


# Data Balancing


In [37]:
# Balance dataset
# Separate classes
spam = df[df['label'] == 1]
non_spam = df[df['label'] == 0]


In [38]:
# Resample to balance
if len(spam) > len(non_spam):
    spam = resample(spam, replace=False, n_samples=len(non_spam), random_state=42)
else:
    non_spam = resample(non_spam, replace=False, n_samples=len(spam), random_state=42)

df = pd.concat([spam, non_spam])


In [39]:
# Combine all tokens into a corpus
corpus = df['tokens'].tolist()


In [40]:
# Build vocabulary
vocab = list(set([word for sentence in corpus for word in sentence]))

In [41]:
# Initialize embeddings
embedding_dim = 10
word_embeddings = {word: np.random.uniform(-1, 1, embedding_dim) for word in vocab}

# Word2Vec Implementation

In [42]:

# Context window size
window_size = 2

# Negative sampling function
def get_negative_samples(vocab, context, num_samples=5):
    negatives = []
    while len(negatives) < num_samples:
        word = random.choice(vocab)
        if word not in context:
            negatives.append(word)
    return negatives


#Model

In [43]:

# Logistic regression with negative sampling
learning_rate = 0.01
epochs = 10

for epoch in range(epochs):
    total_loss = 0
    for sentence in corpus:
        for i, target in enumerate(sentence):
            # Get context words
            start = max(0, i - window_size)
            end = min(len(sentence), i + window_size + 1)
            context = sentence[start:i] + sentence[i+1:end]

            # Negative samples
            negative_samples = get_negative_samples(vocab, context)

            # Training on context words
            for context_word in context:
                # Positive sample
                pos_score = np.dot(word_embeddings[target], word_embeddings[context_word])
                pos_prob = 1 / (1 + np.exp(-pos_score))  # Sigmoid
                pos_loss = -np.log(pos_prob)
                total_loss += pos_loss

                # Gradient update for positive pair
                grad = (1 - pos_prob) * word_embeddings[context_word]
                word_embeddings[target] += learning_rate * grad
                word_embeddings[context_word] += learning_rate * (1 - pos_prob) * word_embeddings[target]

            # Training on negative samples
            for negative_word in negative_samples:
                neg_score = np.dot(word_embeddings[target], word_embeddings[negative_word])
                neg_prob = 1 / (1 + np.exp(-neg_score))  # Sigmoid
                neg_loss = -np.log(1 - neg_prob)
                total_loss += neg_loss

                # Gradient update for negative pair
                grad = -neg_prob * word_embeddings[negative_word]
                word_embeddings[target] += learning_rate * grad
                word_embeddings[negative_word] += learning_rate * (-neg_prob) * word_embeddings[target]

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")



# Save trained embeddings
trained_embeddings = {word: word_embeddings[word] for word in vocab}




# Display sample embeddings
print("\nSample Word Embeddings:")
for word, emb in list(trained_embeddings.items())[:5]:
    print(f"Word: {word}, Embedding: {emb}")


Epoch 1/10, Loss: 1024886.8541
Epoch 2/10, Loss: 902504.3959
Epoch 3/10, Loss: 832643.3766
Epoch 4/10, Loss: 784734.7448
Epoch 5/10, Loss: 752118.1720
Epoch 6/10, Loss: 728173.4481
Epoch 7/10, Loss: 713520.4516
Epoch 8/10, Loss: 702075.1264
Epoch 9/10, Loss: 694746.1105
Epoch 10/10, Loss: 686922.5365

Sample Word Embeddings:
Word: homophobe, Embedding: [ 0.64560227 -0.94316834 -0.59094675 -0.61847938 -1.22804963  0.86028369
  0.10127898  0.44117309 -0.76899321  0.14096299]
Word: 第四大, Embedding: [-0.63106298 -0.32443829  0.56956796  0.28791961 -0.05229969 -0.21771024
  0.43186083  0.07410486  0.07788202 -1.1383926 ]
Word: yielding, Embedding: [-0.52280221  0.1312869   1.00816523 -0.173212    0.29566009 -0.1617118
  0.95786887  0.1978622  -0.56749886  0.13498606]
Word: wdijvlnsjnbnnumberohnumberenumberwnumberytjisxnumberovxsqbnumberssgvthxrrgnumbervnumberrzphhgohyfnumberpoc, Embedding: [-0.49350728  0.03526566  0.87238567 -1.12360528 -0.05495925  0.82427352
 -0.18582713 -0.58358887  0.49

# Saving Embeddings


In [44]:
import pickle

# Save the trained embeddings to a pickle file
with open('trained_embeddings.pkl', 'wb') as file:
    pickle.dump(trained_embeddings, file)

print("Embeddings saved successfully.")

Embeddings saved successfully.


In [47]:
from google.colab import files

# Save the file
with open('trained_embeddings.pkl', 'wb') as file:
    pickle.dump(trained_embeddings, file)

# Download the file to your local machine
files.download('trained_embeddings.pkl')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>